In [98]:
# Imports
import os , glob
import pandas as pd
import requests
import datetime as dt
import numpy as np
import math
import pandas_montecarlo
from dotenv import load_dotenv
from pathlib import Path
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from scipy.stats import norm
import hvplot.pandas
%matplotlib inline

In [2]:
#Read the FMP API Key
load_dotenv()
#Retrive environment variable and store in Python variable
api_key= os.getenv('FMP_API_KEY')
#confirm retrieval of api key
type(api_key)

str

## Define Variables

In [3]:
crypto_symbol = ['BTCUSD','LTCUSD','ETHUSD','XRPUSD']
url = (f'https://financialmodelingprep.com/api/v3/historical-price-full/crypto/{crypto_symbol}?apikey={api_key}')
sp_url = ("https://financialmodelingprep.com/api/v3/historical-price-full/index/%5EGSPC?apikey=b79521d0a15483a6c556d71f892d0be4")
filepath_1 = Path("Data/GC_close_prices.csv")

## Functions for Historical data collection

In [4]:
def get_crypto_data(cryptourl):
    """
        Returns the crypto data from the specified URL as parameter.

    """
    #get companies based on criteria defined about
    CryptoUSD = requests.get(url)
    CryptoUSD = CryptoUSD.json()    
    CryptoUSD = CryptoUSD['historical']
    Crypto = pd.DataFrame.from_dict(CryptoUSD)
    #Crypto['date'] = pd.to_datetime(Crypto['date'])
    Crypto.set_index('date',inplace=True)    
    #Keep only the close column- Open, close, adjclose, volume,chanepercent
    columns =['change','high','label','low','unadjustedVolume','vwap','changeOverTime','close']
    Crypto= Crypto.drop(columns, axis=1)    
    #Rename the column adjclose and changepercent to close and percentchange
    Crypto.rename(columns = {'adjClose' : 'close' , 'changePercent' : 'change%'}, inplace = True)
    return Crypto

In [5]:
def saveto_csv(dataFrame,fileName):    
    newfilename= "Data\\"  + fileName +".csv"    
    dataFrame.to_csv(newfilename,index = True)

In [6]:
def convert_daily_to_monthly(df):
    #Resample dataframe in order to get monthly prices     
    df.index = pd.to_datetime(df.index)
    df=df.resample('M').mean()
    return df

In [7]:
def combine_data(csvFilelist,outputfile):
    i = len(csvFilelist)
    for file in csvFilelist:
        # read first file
        file1 = pd.read_csv("Data\\"+file)
        #read second file
        file2 = pd.read_csv(csvFile2)
        concate_data = pd.concat([file1,file2],index=date)
        concate_data.head()
        print("merged data file",merged_data.head())
    return merged_data  

In [8]:
for symbol in crypto_symbol:
    url = (f'https://financialmodelingprep.com/api/v3/historical-price-full/crypto/{symbol}?apikey={api_key}')
    crypto_df= get_crypto_data(url)
    saveto_csv(crypto_df,symbol)

In [9]:
def get_crypto_returns(csvFile):   
    # set the file path
    file_path = Path(csvFile)
    # create a Pandas DataFrame from a csv file
    df = pd.read_csv(file_path)   
    # drop null values
    df = df.dropna().copy()   
    #set date as index
    df = df.set_index("date")
    # drop all columns except close
    columns =['open','volume']
    df = df.drop(columns, axis=1)
    #rename column
    columnname = symbol[:-3]
    df.rename(columns = {'close' : columnname}, inplace = True)  
    df.rename(columns = {'change%' : columnname+'_change%'}, inplace = True) 
    return df    

In [7]:
def get_sp500_daily():
    sp500daily = requests.get(sp_url)
    sp500daily = sp500daily.json()
    sp500daily = sp500daily['historical']
    sp500_d_change = pd.DataFrame.from_dict(sp500daily)
    sp500_d_change.set_index('date', inplace = True)
    columns =['change','high', 'volume', 'open', 'label','low','unadjustedVolume', 'vwap','changeOverTime', 'close']
    sp500_d_change = sp500_d_change.drop(columns, axis=1)
    sp500_d_change.rename(columns = {'adjClose' : 'close', 'changePercent' : 'change%'}, inplace = True)
    sp500_d_change.to_csv('Data/daily_sp500_returns.csv')
    return sp500_d_change

def get_sp500_monthly():    
    sp500monthly = requests.get(url)
    sp500monthly = sp500monthly.json()
    sp500monthly = sp500monthly['historical']
    sp_m_change = pd.DataFrame.from_dict(sp500monthly)
    columns =['change','high','label','low','unadjustedVolume', 'vwap','changeOverTime', 'close']    
    sp_m_change = sp_m_change.drop(columns, axis=1)
    sp_m_change.rename(columns = {'adjClose' : 'close', 'changePercent' : 'change%'}, inplace = True)
    sp_m_change.set_index('date', inplace=True)
    sp_m_change.index = pd.to_datetime(sp_m_change.index)
    sp_m_change = sp_m_change.resample('1M').mean()
    sp_m_change.to_csv('Data/monthly_sp500_returns.csv')    
    return sp_m_change

def get_sp_data():     
    get_sp500_daily()
    get_sp500_monthly()
    return

In [ ]:
#change DateTime Format
def get_gold_data()    
    gold_df = pd.read_csv(filepath_1,index_col="Date",parse_dates=True,infer_datetime_format=True)
    gold_df.reset_index()
    gold_df['Date']= pd.to_datetime(gold_df['Date'])
    gold_df['Date'] = gold_df['Date'].dt.strftime("%Y/%m/%d")
    #Set Index
    gold_df.set_index(pd.to_datetime(gold_df['Date'], infer_datetime_format=True), inplace=True)
    gold_df.drop(columns=['Date'], inplace=True)
    gold_df.rename(columns={'Close/Last': 'Gold_Close'},inplace = True)
    return gold_df

In [ ]:
def gold_returns()
    #Calculate daily Gold returns
    gold_returns = get_gold_data()
    gold_returns.drop(columns=['Open','High','Volume','Low'], inplace=True)
    gold_returns = gold_returns.pct_change()
    #Rename columns
    gold_returns.rename(columns={'Gold_Close':'Gold_Rate%'},inplace = True)
    return gold_returns

In [ ]:
def get_silver_data()    
    # Reading Silver returns
    filepath_2 = Path("Data/SI_close_prices.csv")
    silver_df = pd.read_csv(filepath_2)
    #Change DateTime Format
    silver_df['Date']= pd.to_datetime(silver_df['Date'])
    silver_df['Date']= silver_df['Date'].dt.strftime("%Y/%m/%d")
    #Set Index
    silver_df.set_index(pd.to_datetime(silver_df['Date'], infer_datetime_format=True), inplace=True)
    silver_df.rename(columns={'Close/Last': 'Silver_Close'},inplace = True)
    silver_df.drop(columns=['Date'], inplace=True)
    return silver_df

In [ ]:
def silver_returns()
    #Calculate daily Silver returns
    silver_returns = get_silver_data()
    silver_returns.drop(columns=['Open','High','Volume','Low'], inplace=True)
    silver_returns = silver_returns.pct_change()
    #Rename columns
    silver_returns.rename(columns={'Silver_Close':'Silver_Rate%'},inplace = True)
    return silver_returns

In [ ]:
def get_treaury_data()
    Daily_US_Treasury_Yield = Path('Data/US Daily Yield.csv')
    Daily_US_Treasury_Yield_DF = pd.read_csv(Daily_US_Treasury_Yield, index_col = "Date", parse_dates=True, infer_datetime_format=True)
    Daily_US_Treasury_Yield_DF.head(10)
    five_year = Daily_US_Treasury_Yield_DF['5 Yr']
    five_year_df = pd.DataFrame(data = five_year)
    five_year_df.plot.line(figsize = (15,7), title = "Daily Yield Changes in US Treasury Bonds, 2015 - 2020")
    return five_year_df

## Data Cleanup Process

In [ ]:
get_sp_data()

In [10]:
for symbol in crypto_symbol: 
    csvFile = 'Data\\'+symbol+'.csv'
    if symbol == 'BTCUSD':
        btc_df=get_crypto_returns(csvFile)       
    elif symbol == 'LTCUSD':
        ltc_df=get_crypto_returns(csvFile)
    elif symbol == 'XRPUSD':
        xrp_df=get_crypto_returns(csvFile)
    elif symbol == 'ETHUSD':
        etc_df=get_crypto_returns(csvFile)    

combine_crypto_df = pd.concat([btc_df,ltc_df,xrp_df,etc_df], axis = 'columns', join='inner')
saveto_csv(combine_crypto_df,'daily_cryptoall')
monthly_crypto_df=convert_daily_to_monthly(combine_crypto_df)
saveto_csv(monthly_crypto_df,'monthly_cryptoall')

In [11]:
csvFilelist=['daily_cryptoall.csv','daily_commodity_df.csv','daily_sp500_returns.csv','daily_bond_data.csv']
i = len(csvFilelist)
for file in csvFilelist:
    if file.startswith('daily_crypto'):
        file1 = pd.read_csv("Data\\"+file, parse_dates=True, infer_datetime_format=True)
        file1=file1.set_index('date')
    elif file.startswith('daily_sp500'):
        #read second file    
        file2 = pd.read_csv("Data\\"+file,parse_dates=True, infer_datetime_format=True)         
        file2=file2.set_index('date')
        file2.rename(columns = {'close' : 'sp500_close','change%' :'sp500_change%'}, inplace = True)        
    elif file.startswith('daily_commodity'):
        file3 = pd.read_csv("Data\\"+file, parse_dates=True, infer_datetime_format=True)
        # rename Pandas columns to lower case        
        file3.columns= file3.columns.str.lower()
        file3 = file3.drop(['open','low','high','volume','open.1','low.1','high.1','volume.1'],axis=1)
        file3['date'] = pd.to_datetime(file3['date'], format="%d/%m/%Y")                      
        file3=file3.set_index('date')       
    else:
        file4=pd.read_csv("Data\\"+file,parse_dates=True, infer_datetime_format=True)                
        file4.columns= file4.columns.str.lower()     
        file4['5 yr'] = file4['5 yr'] / 100
        file4=file4.set_index('date')        
        file4.rename(columns = {'5 yr' : 'bonds5yr_close'}, inplace = True)           
    

concate_data = pd.merge(file1,file2, on='date', how='inner')
merged_df = concate_data.join(file3, how='inner')
final_merged_df = merged_df.join(file4, how='inner')
saveto_csv(final_merged_df,'daily_combineall')     

In [12]:
csvFilelist=['monthly_cryptoall.csv','monthly_commodity_df.csv','monthly_sp500_returns.csv','monthly_bonds_data.csv']
i = len(csvFilelist)
for file in csvFilelist:
    if file.startswith('monthly_crypto'):
        file1 = pd.read_csv("Data\\"+file, parse_dates=True, infer_datetime_format=True)
        file1=file1.set_index('date')
    elif file.startswith('monthly_sp500'):
        #read second file    
        file2 = pd.read_csv("Data\\"+file, parse_dates=True, infer_datetime_format=False)               
        file2 = file2.drop(['open','volume'],axis=1)
        file2.rename(columns = {'close' : 'sp500_close', 'change%' : 'sp500_change%'}, inplace = True)
        file2=file2.set_index('date')
    elif file.startswith('monthly_commodity'):
        file3 = pd.read_csv("Data\\"+file, parse_dates=True, infer_datetime_format=True)
        # rename Pandas columns to lower case        
        file3.columns= file3.columns.str.lower()
        file3 = file3.drop(['open','low','high','open.1','low.1','high.1'],axis=1)
        #file3['date'] = pd.to_datetime(file3['date'], format="%d/%m/%Y")   
        file3=file3.set_index('date')
    else:
        file4=pd.read_csv("Data\\"+file,parse_dates=True, infer_datetime_format=True)
        file4.columns= file4.columns.str.lower()
        file4=file4.set_index('date')
        file4.rename(columns = {'rate' : 'bonds_close'}, inplace = True)    
    

concate_monthly = pd.concat([file1,file2,file3,file4],axis='columns', join='inner')
saveto_csv(concate_monthly,'monthly_combineall')     